## 1. 데이터 불러오기
[캐글 사이트](https://www.kaggle.com/datasets/zygmunt/goodbooks-10k)

In [2]:
import numpy as np
import pandas as pd
import os

In [3]:
print(os.listdir('../data/book/'))

['books.csv', 'book_ratings.csv', 'book_tags.csv', 'sample_book.xml', 'tags.csv', 'to_read.csv']


In [4]:
books = pd.read_csv('../data/book/books.csv', encoding='ISO-8859-1')
books.head()

,id,book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,...,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...
1,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPrÃ©",1997.0,Harry Potter and the Philosopher's Stone,...,4602479,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...
2,3,41865,41865,3212258,226,316015849,9.780316e+12,Stephenie Meyer,2005.0,Twilight,...,3866839,3916824,95009,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...
3,4,2657,2657,3275794,487,61120081,9.780061e+12,Harper Lee,1960.0,To Kill a Mockingbird,...,3198671,3340896,72586,60427,117415,446835,1001952,1714267,https://images.gr-assets.com/books/1361975680m...,https://images.gr-assets.com/books/1361975680s...
4,5,4671,4671,245494,1356,743273567,9.780743e+12,F. Scott Fitzgerald,1925.0,The Great Gatsby,...,2683664,2773745,51992,86236,197621,606158,936012,947718,https://images.gr-assets.com/books/1490528560m...,https://images.gr-assets.com/books/1490528560s...


- ratings_1	ratings_2	ratings_3	ratings_4	ratings_5	 : 별 1,2,3,4,5 개 준 사람들 수
- 5개 만점

In [5]:
ratings = pd.read_csv('../data/book/book_ratings.csv', encoding='ISO-8859-1')
ratings.head()

,book_id,user_id,rating
0,1,314,5
1,1,439,3
2,1,588,5
3,1,1169,4
4,1,1185,4


In [9]:
book_tags = pd.read_csv('../data/book/book_tags.csv', encoding='ISO-8859-1')
book_tags.head()

,goodreads_book_id,tag_id,count
0,1,30574,167697
1,1,11305,37174
2,1,11557,34173
3,1,8717,12986
4,1,33114,12716


In [10]:
tags = pd.read_csv('../data/book/tags.csv')
tags.head()

,tag_id,tag_name
0,0,-
1,1,--1-
2,2,--10-
3,3,--12-
4,4,--122-


In [12]:
to_read = pd.read_csv('../data/book/to_read.csv')
to_read.head()

,user_id,book_id
0,1,112
1,1,235
2,1,533
3,1,1198
4,1,1874


---
## 2. 데이터 전처리

### book_tags 와 tags를 merge

In [11]:
tags_join_df = pd.merge(book_tags,tags, left_on='tag_id', right_on='tag_id', how='inner')

tags_join_df.head()

,goodreads_book_id,tag_id,count,tag_name
0,1,30574,167697,to-read
1,2,30574,24549,to-read
2,3,30574,496107,to-read
3,5,30574,11909,to-read
4,6,30574,298,to-read


---
## 3. 유사도 측정

In [17]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
cosine_sim

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [16]:
from sklearn.metrics.pairwise import linear_kernel

---
## 4 . 작가이름을 기준으로 비슷한 책 추천
- Tfidf를 book의 'author'로 수행

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [13]:
books['authors'][:5]

0                 Suzanne Collins
1    J.K. Rowling, Mary GrandPrÃ©
2                 Stephenie Meyer
3                      Harper Lee
4             F. Scott Fitzgerald
Name: authors, dtype: object

In [15]:
tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 2), min_df=0, stop_words='english')

tfidf_matrix = tf.fit_transform(books['authors'])
tfidf_matrix

<10000x14742 sparse matrix of type '<class 'numpy.float64'>'
	with 43235 stored elements in Compressed Sparse Row format>

### 인덱스 값

In [19]:
titles = books['title']
indices = pd.Series(books.index, index=books['title'])
indices['The Hobbit']

6

### 유사도 값

In [21]:
cosine_sim[6]

array([0., 0., 0., ..., 0., 0., 0.])

### 유사도 결과를 인덱스 있는 list 형으로

In [22]:
cosine_sim[6].shape

(10000,)

In [23]:
list(enumerate(cosine_sim[6]))[:3]

[(0, 0.0), (1, 0.0), (2, 0.0)]

### 가장 유사한 책의 인덱스 찾기

In [25]:
sim_scores = list(enumerate(cosine_sim[6]))
sim_scores = sorted(sim_scores, key = lambda x : x[1], reverse=True)
sim_scores[:5]

[(6, 1.0), (18, 1.0), (154, 1.0), (160, 1.0), (188, 1.0)]

### 작가로 본 유사 책 검색

In [27]:
sim_scores = sim_scores[1:6] # 자기 자신 제외

book_indices = [i[0] for i in sim_scores]
titles.iloc[book_indices]

154            The Two Towers (The Lord of the Rings, #2)
160     The Return of the King (The Lord of the Rings,...
188     The Lord of the Rings (The Lord of the Rings, ...
963     J.R.R. Tolkien 4-Book Boxed Set: The Hobbit an...
4975        Unfinished Tales of NÃºmenor and Middle-Earth
Name: title, dtype: object

---
## 5. tag를 기준으로 유사한 책 추전
- Tfidf를 books_with_tags의 'tag'로 수행

###  book에 tag 포함

In [29]:
books_with_tags = pd.merge(books, tags_join_df, left_on='book_id', right_on='goodreads_book_id', how='inner')
books_with_tags.head()

,id,book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url,goodreads_book_id,tag_id,count,tag_name
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,...,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...,2767052,30574,11314,to-read
1,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,...,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...,2767052,11305,10836,fantasy
2,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,...,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...,2767052,11557,50755,favorites
3,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,...,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...,2767052,8717,35418,currently-reading
4,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,...,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...,2767052,33114,25968,young-adult


In [30]:
tf1 = TfidfVectorizer(analyzer='word', ngram_range=(1, 2), min_df=0, stop_words='english')

tfidf_matrix1 = tf1.fit_transform(books_with_tags['tag_name'].head(10000))
cosine_sim1= linear_kernel(tfidf_matrix1, tfidf_matrix1)

In [31]:
titles1 = books['title']
indices1 = pd.Series(books.index, index=books['title'])

### tag를 기반으로 유사한 책을 찾아주는 함수

In [32]:
def tags_recom(title):
    idx = indices1[title]
    
    sim_scores = list(enumerate(cosine_sim1[idx]))
    sim_scores = sorted(sim_scores, key = lambda x : x[1], reverse=True)
    sim_scores = sim_scores[1:11]
    
    book_indices = [i[0] for i in sim_scores]
    
    return titles.iloc[book_indices]

### "THe Hobbit"

In [33]:
tags_recom('The Hobbit')

16             Catching Fire (The Hunger Games, #2)
31                                  Of Mice and Men
107    Confessions of a Shopaholic (Shopaholic, #1)
125                       Dune (Dune Chronicles #1)
149                                    The Red Tent
206          One for the Money (Stephanie Plum, #1)
214                                Ready Player One
231             The Gunslinger (The Dark Tower, #1)
253          Shiver (The Wolves of Mercy Falls, #1)
313                         Inkheart (Inkworld, #1)
Name: title, dtype: object

---
## 6. 작가명 & 태그를 기준으로 비슷한 책 추천

### (1) 데이터 재정비

### 임시로 book id 마다 tag 붙이기

In [35]:
temp_df = books_with_tags.groupby('book_id')['tag_name'].apply(' '.join).reset_index()
temp_df

,book_id,tag_name
0,1,to-read fantasy favorites currently-reading yo...
1,2,to-read fantasy favorites currently-reading yo...
2,3,to-read fantasy favorites currently-reading yo...
3,5,to-read fantasy favorites currently-reading yo...
4,6,to-read fantasy young-adult fiction harry-pott...
...,...,...
9995,31538647,to-read fantasy favorites currently-reading yo...
9996,31845516,to-read favorites currently-reading fiction bo...
9997,32075671,to-read favorites currently-reading young-adul...
9998,32848471,to-read favorites currently-reading fiction bo...


### 위 내용을 books와 합치기

In [36]:
books = pd.merge(books, temp_df, left_on='book_id', right_on='book_id', how='inner')
books.head()

,id,book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url,tag_name
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,...,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...,to-read fantasy favorites currently-reading yo...
1,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPrÃ©",1997.0,Harry Potter and the Philosopher's Stone,...,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...,to-read fantasy favorites currently-reading yo...
2,3,41865,41865,3212258,226,316015849,9.780316e+12,Stephenie Meyer,2005.0,Twilight,...,3916824,95009,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...,to-read fantasy favorites currently-reading yo...
3,4,2657,2657,3275794,487,61120081,9.780061e+12,Harper Lee,1960.0,To Kill a Mockingbird,...,3340896,72586,60427,117415,446835,1001952,1714267,https://images.gr-assets.com/books/1361975680m...,https://images.gr-assets.com/books/1361975680s...,to-read favorites currently-reading young-adul...
4,5,4671,4671,245494,1356,743273567,9.780743e+12,F. Scott Fitzgerald,1925.0,The Great Gatsby,...,2773745,51992,86236,197621,606158,936012,947718,https://images.gr-assets.com/books/1490528560m...,https://images.gr-assets.com/books/1490528560s...,to-read favorites currently-reading young-adul...


### 저자명과 태그 합치기

In [39]:
pd.Series(books[['authors', 'tag_name']].fillna('').values.tolist())

0       [Suzanne Collins, to-read fantasy favorites cu...
1       [J.K. Rowling, Mary GrandPrÃ©, to-read fantasy...
2       [Stephenie Meyer, to-read fantasy favorites cu...
3       [Harper Lee, to-read favorites currently-readi...
4       [F. Scott Fitzgerald, to-read favorites curren...
                              ...                        
9995    [Ilona Andrews, to-read fantasy favorites curr...
9996    [Robert A. Caro, to-read favorites currently-r...
9997    [Patrick O'Brian, to-read favorites currently-...
9998    [Peggy Orenstein, to-read favorites currently-...
9999    [John Keegan, to-read favorites currently-read...
Length: 10000, dtype: object

In [37]:
books['corpus'] = (pd.Series(books[['authors', 'tag_name']].fillna('').values.tolist()).str.join(' '))
books['corpus'][:3]

0    Suzanne Collins to-read fantasy favorites curr...
1    J.K. Rowling, Mary GrandPrÃ© to-read fantasy f...
2    Stephenie Meyer to-read fantasy favorites curr...
Name: corpus, dtype: object

### (2) Tfidf 실행

In [41]:
tf_corpus = TfidfVectorizer(analyzer='word', ngram_range=(1, 2), min_df=0, stop_words='english')
tfidf_matrix_corpus = tf_corpus.fit_transform(books['corpus'])
cosine_sim_corpus = linear_kernel(tfidf_matrix_corpus, tfidf_matrix_corpus)

titles = books['title']
indices = pd.Series(books.index, index=books['title'])

### 추천함수

In [42]:
def corpus_recom(title):
    idx = indices1[title]
    
    sim_scores = list(enumerate(cosine_sim_corpus[idx]))
    sim_scores = sorted(sim_scores, key = lambda x : x[1], reverse=True)
    sim_scores = sim_scores[1:11]
    
    book_indices = [i[0] for i in sim_scores]
    
    return titles.iloc[book_indices]

### "The Hobbit"

In [43]:
corpus_recom("The Hobbit")

188     The Lord of the Rings (The Lord of the Rings, ...
154            The Two Towers (The Lord of the Rings, #2)
160     The Return of the King (The Lord of the Rings,...
18      The Fellowship of the Ring (The Lord of the Ri...
610              The Silmarillion (Middle-Earth Universe)
4975        Unfinished Tales of NÃºmenor and Middle-Earth
2308                               The Children of HÃºrin
963     J.R.R. Tolkien 4-Book Boxed Set: The Hobbit an...
465                             The Hobbit: Graphic Novel
8271                   The Complete Guide to Middle-Earth
Name: title, dtype: object